In [ ]:
import qiskit
import qiskit.qasm3
import openqasm3
import QSymPy
import numpy as np
import tqdm
import sympy.core

In [ ]:
qc = qiskit.QuantumCircuit(2)
qc.h(0)
qc.barrier()
qc.cx(0, 1)
qc.barrier()
qc.measure_all()

In [ ]:
qreg1 = qiskit.QuantumRegister(4,'qqqq')
qc = qiskit.QuantumCircuit(qreg1)
qc.h(qreg1)
qc.u(np.pi/2, np.pi/4, np.pi/8, qreg1[0])
qc.cx(qreg1[0], qreg1[1])


In [ ]:
with open('test.qasm', 'w') as f:
    qiskit.qasm3.dump(qc, f)

In [ ]:
with open ('test.qasm', 'r') as f:
#with open ('HHL_circuit_Qiskit_2x2.qasm3', 'r') as f:
    qasm = f.read()

print(qasm)

In [ ]:
qc = QSymPy.openqasm3_to_qc('test.qasm')
#qc = QSymPy.openqasm3_to_qc('HHL_circuit_Qiskit_2x2.qasm3')

In [ ]:
print(qc.gate_collection.collections['H'])
print(qc.gate_collection.collections['U'])
print(qc.gate_collection.collections['X'])
print(qc.qubits)
for k,v in qc.gate_collection.collections.items():
    for gate in v:
        if gate.step == 0 and k in ('X',):
            pass
            print(gate.name, gate.qubits_t, gate.qubits_c, gate.parameters)
        else:
            pass
            #print(gate.name, gate.qubits_t, gate.qubits_c, gate.parameters)
            gate.treat_numeric_only = True


In [ ]:
type(qc.gate_collection.collections['H'][0].atomics['00'])

In [ ]:
qc.assemble_symbolic_unitary(use_alternative_repr=True, replace_symbolic_zeros_and_ones=False)
#qc.unitary

In [ ]:
a = qc.gate_collection.collections['H'][0].atomics['01']
a

In [ ]:
i = 0
for e in tqdm.tqdm(sympy.core.traversal.postorder_traversal(qc.unitary)):
    i += 1
print(i)

In [ ]:
import sympy
import sympy.core
import sympy.core.backend
dir(sympy.core.backend)
type(sympy.core.backend.diff)

In [ ]:
%timeit sympy.core.backend.diff(qc.unitary, a)

In [ ]:
%timeit qc.unitary.diff(a)

In [ ]:
qc.unitary.free_symbols

In [ ]:
qc.unitary

In [ ]:
dict_replacements = {}
for gate_type, gates in qc.gate_collection.collections.items():
    #print(gate_type, len(gates))
    for g in gates:
        for q_t in g.qubits_t:
            for i in range(len(g.matrix22_t[q_t])):
                for symb, num in zip(g.matrix22_t[q_t][i].flat(), g.matrix22_t_numeric[q_t][i].flatten()):
                    if num == 0:
                        dict_replacements[symb] = 0
                    elif num == 1:
                        dict_replacements[symb] = 1
dict_replacements

In [ ]:
qc.assemble_symbolic_unitary(use_alternative_repr=True, replace_symbolic_zeros_and_ones=True)
#print(qc.unitary)

In [ ]:
#new_unitary = qc.unitary.xreplace(dict_replacements)

In [ ]:
type(qc.unitary)
print(qc.unitary.shape)


In [ ]:
state = QSymPy.QuantumState(num_qubits=len(qc.qubits))
state.set_state({k:0 for k in range(len(qc.qubits))})

In [ ]:
state2 = qc.unitary @ state.state

In [ ]:
a = qc.gate_collection.collections['CU'][0].matrix22_t_alt[1][1][0,0]
print(a)
b = a.args[0].args[0].args[1]
b

In [ ]:
#grad_state2_32 = sympy.diff(state2[32], b)


In [ ]:
i = 0
for e in tqdm.tqdm(sympy.core.traversal.postorder_traversal(state2[32])):
    i += 1
print(i)

In [ ]:
a = qc.gate_collection.collections['CU'][0].matrix22_t[1][1][0,0]
grad_a = QSymPy.sp.diff(state2, a)

In [ ]:
qc.create_numeric_unitary_from_symbolic()
print(qc.unitary_numeric)

In [ ]:
qasm_ast_program = openqasm3.ast.Program(qasm)
#openqasm3.parse(qasm)

In [ ]:
qasm_ast_program.span

In [ ]:
qasm_parsed_file= openqasm3.parse(qasm)

In [ ]:
qasm_parsed_file.span

In [ ]:
for s in qasm_parsed_file.statements:
    print(s)

In [ ]:
qasm_parsed_file.version

In [ ]:
state = openqasm3.visitor.QASMVisitor().visit(node=qasm_parsed_file)

In [ ]:
state

In [ ]:
qasm_parsed_file.statements

In [ ]:
for statement in qasm_parsed_file.statements:
    print(statement)

In [ ]:
import QSymPy

In [ ]:
qcs = QSymPy.QuantumCircuit(num_qubits=2, num_clbits=2)
qcs.add_gate('H', qubits_t=[0], step=0)
qcs.add_barrier(step=0)
qcs.add_gate('CX', qubits_c=[0], qubits_t=[1], step=1)
qcs.add_barrier(step=1)

qcs.assemble_symbolic_unitary()
qcs.unitary


In [ ]:
for e in qcs.unitary[0,0].free_symbols: break
e.name

In [ ]:
qcs.barrier_collection

In [ ]:
qcs2 = QSymPy.QuantumCircuit(num_qubits=2, num_clbits=2)
qcs2.add_gate(name='U', qubits_t=[0], step=0, parameters={'theta': 0.5, 'phi': 0.5, 'lambda': 0.5})
qcs2.add_gate(name='X', qubits_t=[1], step=0)
qcs2.assemble_symbolic_unitary()
qcs2.unitary

In [ ]:
qcs2.gate_collection.collections['U'][0].matrix_alt

In [ ]:
#subs_symbolic_alternatives_in_symbolic_unitary
qcs2.unitary

In [ ]:
#qcs2.subs_symbolic_alternatives_in_symbolic_unitary({'steps': (0,0), 'names': ('U','U'), 'qubits_t': (0,1)})
qcs2.subs_symbolic_alternatives_in_symbolic_unitary()
qcs2.unitary

In [ ]:
qcs3 = QSymPy.QuantumCircuit(num_qubits=2, num_clbits=2)
qcs3.add_gate(name='GP', qubits_t=[0], step=0, parameters={'gamma': 0.5})
qcs3.assemble_symbolic_unitary()
qcs3.unitary

In [ ]:
qcs3.subs_symbolic_alternatives_in_symbolic_unitary()
qcs3.unitary

In [ ]:
qcs4 = QSymPy.QuantumCircuit(num_qubits=2, num_clbits=2)
qcs4.add_gate(name='P', qubits_t=[0], step=0, parameters={'lambda': 0.5})
qcs4.assemble_symbolic_unitary()
qcs4.unitary

In [ ]:
qcs4.subs_symbolic_alternatives_in_symbolic_unitary()
qcs4.unitary

In [ ]:
import numpy as np
qcs2.gate_collection.collections['U'][0].matrix_numeric


In [ ]:
np.array(qcs2.gate_collection.collections['U'][0].matrix_numeric)

In [ ]:
qcs2.gate_collection.collections['U'][0].atomics

In [ ]:
qcs2.subs_symbolic_zerosones_in_symbolic_unitary()
qcs2.unitary

In [ ]:
qcs2.create_numeric_unitary_from_symbolic()
qcs2.unitary_numeric

In [ ]:
qc_imported = QSymPy.openqasm3_to_qc('test.qasm')
#qc_imported = QSymPy.openqasm3_to_qc('HHL_circuit_Qiskit_2x2.qasm3')

In [ ]:
qc_imported.assemble_symbolic_unitary()
qc_imported.unitary

In [ ]:
qcs4 = QSymPy.QuantumCircuit(num_qubits=3, num_clbits=2)
qcs4.add_gate(name='CX', qubits_t=[0], qubits_c=[1], step=0)
qcs4.assemble_symbolic_unitary()
qcs4.unitary

In [ ]:
qcs4.gate_collection.collections['CX'][0].matrix

In [ ]:
qcs4.gate_collection.collections['CX'][0].atomics

In [ ]:
qcs4.assemble_symbolic_unitary()
qcs4.unitary

In [ ]:
qcs4.subs_symbolic_zerosones_in_symbolic_unitary()
qcs4.unitary

In [ ]:
qcs4.unitary.atoms()

In [ ]:
qcs4.unitary.subs({'CX_X_qt0_qc1_s0_p01': -1})

In [ ]:
qcs4.create_numeric_unitary_from_symbolic()
qcs4.unitary_numeric

In [ ]:
qcs5 = QSymPy.QuantumCircuit(num_qubits=3, num_clbits=2)
qcs5.add_gate(name='CCX', qubits_t=[0], qubits_c=[1, 2], step=0)
qcs5.assemble_symbolic_unitary()
qcs5.unitary

In [ ]:
qcs5.create_numeric_unitary_from_symbolic()
qcs5.unitary_numeric

In [ ]:
qcs5.gate_collection.collections['CCX'][0].matrix

In [ ]:
qcs6 = QSymPy.QuantumCircuit(num_qubits=4, num_clbits=2)
qcs6.add_gate(name='CCXX', qubits_t=[0,1], qubits_c=[2, 3], step=0)
qcs6.assemble_symbolic_unitary()
qcs6.unitary

In [ ]:
qcs6.create_numeric_unitary_from_symbolic()
qcs6.unitary_numeric

In [ ]:
qcs7 = QSymPy.QuantumCircuit(num_qubits=2, num_clbits=2)
qcs7.add_gate(name='CU', qubits_t=[0], qubits_c=[1], step=0, parameters={'theta': 0.5, 'phi': 0.5, 'lambda': 0.5})
qcs7.assemble_symbolic_unitary()
qcs7.unitary

In [ ]:
qcs7.subs_symbolic_alternatives_in_symbolic_unitary()
qcs7.unitary

In [ ]:
qcs8 = QSymPy.QuantumCircuit(num_qubits=3, num_clbits=2)
qcs8.add_gate(name='CUU', qubits_t=[0,1], qubits_c=[2], step=0, parameters=[{'theta': 0.5, 'phi': 0.5, 'lambda': 0.5}, {'theta': 0.5, 'phi': 0.5, 'lambda': 0.5}])
qcs8.assemble_symbolic_unitary()
qcs8.unitary

In [ ]:
qcs8.subs_symbolic_alternatives_in_symbolic_unitary()
qcs8.unitary

In [ ]:
qcs8.subs_symbolic_zerosones_in_symbolic_unitary(zeros=True, ones=True)
qcs8.unitary